In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link = 'https://drive.google.com/open?id=1Q8KUlytiQ2UDI6qb0YML8vASLDAijDg2'
_, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
print(downloaded)
downloaded.GetContentFile('split_ori_face.zip')

In [ ]:
!ls
# !unzip split_ori_face.zip

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!unzip split_ori_face.zip

In [ ]:
def make_weights_for_balanced_classes(images, nclasses):                        
    count = [0] * nclasses                                                      
    for item in images:                                                         
        count[item] += 1                                                     
    weight_per_class = [0.] * nclasses                                      
    N = float(sum(count))                                                   
    for i in range(nclasses):   
        if count[i] != 0:
          weight_per_class[i] = N/float(count[i])                                 
    weight = [0] * len(images)                                              
    for idx, val in enumerate(images):                                          
        weight[idx] = weight_per_class[val]                                  
    return weight 

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.sampler import WeightedRandomSampler
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as func
import torch.nn.init as torch_init
import torch.optim as optim
import pandas as pd
import torchvision
from torchvision import transforms, utils
import os
from sklearn.model_selection import KFold
import copy
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore")

class loader(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.transform = transform
        self.frame = pd.read_csv(csv_file)
        self.root_dir = root_dir

    def __len__(self):
        return len(self.frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,self.frame.iloc[idx, 0])
        img = Image.open(img_name)
        label = self.frame.iloc[idx, 1]
        
        if self.transform:
            img = self.transform(img)

        return img, label
    
    
class Nnet(nn.Module):
    def __init__(self):
        super(Nnet, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 21 , 3, stride=2, padding=1, bias=False), 
            nn.ReLU(inplace=True),
            nn.Conv2d(21, 20, 3,stride=2, padding=1, bias=False),
            nn.BatchNorm2d(num_features=20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(20, 15, 3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(num_features=15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU( inplace=True),
            nn.Conv2d(15, 7, 5, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(num_features=7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU( inplace=True),
        )
        self.fc = nn.Sequential(
            nn.Linear(1183,500),
            nn.ReLU( inplace=True),
            nn.Linear(500, 201),
            #nn.Softmax()
        )

    def num_flat_features(self, inputs):
        
        # Get the dimensions of the layers excluding the inputs
        size = inputs.size()[1:]
        # Track the number of features
        num_features = 1
        
        for s in size:
            num_features *= s
        
        return num_features

    def forward(self, input):
        x=self.main(input)
        #print('yo:', self.num_flat_features(x))
        x=x.view(-1, self.num_flat_features(x))
        
        return self.fc(x)

        
os.environ['XDG_CACHE_HOME']='/tmp/xdg-cache'

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

# Check if your system supports CUDA
use_cuda = torch.cuda.is_available()

# Setup GPU optimization if CUDA is supported
if use_cuda:
    computing_device = torch.device("cuda")
    extras = {"num_workers": 1, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    computing_device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")

net=Nnet().to(computing_device)
net.apply(weights_init)

# Print the model
print(net)

#loss criteria are defined in the torch.nn package
criterion = nn.CrossEntropyLoss()


#Instantiate the gradient descent optimizer - use Adam optimizer with default parameters
optimizer = optim.Adam(net.parameters(),lr = 0.001)

transform = transforms.Compose([transforms.Resize(224), transforms.CenterCrop(224), transforms.ToTensor()])
dataset = loader('train.csv','/content/',transform=transform)
batch_size = 64
validation_split = .33
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler)

# Track the loss across training
total_loss = []
t_loss = []
t_acc = []
v_loss = []
models = []
optims = []
folds_acc = []
best_error = np.inf
best_accuracy = 0
avg_minibatch_loss = []
N = 50
early_stop_num = 5
es_count = 0

for epoch in range(20):
    N_minibatch_loss = 0.0
    correct = 0
    total = 0


    # Get the next minibatch of images, labels for training
    for minibatch_count, (images, labels) in enumerate(train_loader, 0):
        #print("mini_batch", minibatch_count)
        # Zero out the stored gradient (buffer) from the previous iteration
        optimizer.zero_grad()
        # Put the minibatch data in CUDA Tensors and run on the GPU if supported
        images, labels = images.to(computing_device), labels.to(computing_device)
        # Perform the forward pass through the network and compute the loss
        outputs = net(images)
        
        loss = criterion(outputs, labels)
        # Automagically compute the gradients and backpropagate the loss through the network
        loss.backward()

        # Update the weights
        optimizer.step()    
        # Add this iteration's loss to the total_loss
        total_loss.append(loss.item())
        N_minibatch_loss += loss
               
        
        if minibatch_count % N == 49:
            #Print the loss averaged over the last N mini-batches
            N_minibatch_loss /= N
            print('Epoch %d, average minibatch %d loss: %.3f' % (epoch + 1, minibatch_count+1, N_minibatch_loss))
            # Add the averaged loss over N minibatches and reset the counter
            avg_minibatch_loss.append(N_minibatch_loss)
            N_minibatch_loss = 0.0

    print("Finished", epoch + 1, "epochs of training")
    # TODO: Implement validation #with torch.no_grad():
    
    with torch.no_grad():
        for minibatch_count, (images, labels) in enumerate(validation_loader, 0):
            # Put the minibatch data in CUDA Tensors and run on the GPU if supported
            images, labels = images.to(computing_device), labels.to(computing_device)
            for index in range(len(labels)):    
                real_y = labels[index]
                #print('real_y', real_y)
                out = net(images[index].view(-1,3,224,224))
                #print('out:', out)
                pred_y = torch.argmax(out)
                #print('pred_y', pred_y)

                if (real_y.item() == pred_y.item()):
                    correct += 1
                total += 1
    accuracy = correct/total
    val_error = 1-accuracy
        
    # Early stop checking
    if val_error >= best_error:
        es_count += 1
        if es_count == early_stop_num:
            print('breaking')
            break
    else:
        es_count = 0 
        best_error = val_error
        best_accuracy = accuracy
        best_model = copy.deepcopy(net.state_dict())
        best_optim = copy.deepcopy(optimizer.state_dict())
          
        
    t_loss.append(np.mean(total_loss))
    t_acc.append(100-(np.mean(total_loss)))
    v_loss.append(val_error) 
    print('Iteration accuracy: %.3f' % ((accuracy)*100))
    print('Val Error: %.3f' %((val_error)*100))
    
# Best model for kth fold
models.append(best_model)
optims.append(best_optim)
print('Best accuracy: %.3f' % ((best_accuracy)*100))
folds_acc.append(best_accuracy)
#kfold += 1

# Getting best model
best_idx = np.argmax(folds_acc)
best_model = models[best_idx]
best_optim = optims[best_idx]
# Loading
net.load_state_dict(best_model)
optimizer.load_state_dict(best_optim)

# Using whole dataset as test set
transform = transforms.Compose([transforms.Resize(224), transforms.CenterCrop(224), transforms.ToTensor()])
testset = loader('test.csv','/content/',transform=transform)
testset_size = len(testset)
indices = list(range(testset_size))
#split = int(np.floor(validation_split * dataset_size))
#train_indices, val_indices = indices[split:], indices[:split]
valid_sampler = SubsetRandomSampler(indices)
validation_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                                    sampler=valid_sampler)

y_vals = []
y_preds = []
with torch.no_grad():
    for minibatch_count, (images, labels) in enumerate(validation_loader, 0):
        # Put the minibatch data in CUDA Tensors and run on the GPU if supported
        images, labels = images.to(computing_device), labels.to(computing_device)
        for index in range(len(labels)):    
            real_y = labels[index]
            y_vals.append(real_y.item())
            #print('real_y', real_y.item())
            out = net(images[index].view(-1,3,224,224))
            #print('out:', out)
            pred_y = torch.argmax(out)
            y_preds.append(pred_y.item())
            #print('pred_y', pred_y.item())

            if (real_y.item() == pred_y.item()):
                correct += 1
            total += 1
    accuracy = correct/total

print('tloss', t_loss)
print('t_acc', t_acc)
print('vloss:', v_loss)
print('v_acc:', folds_acc)
report = classification_report(y_vals, y_preds, labels=range(201), output_dict=True)
conf_matrix = confusion_matrix(y_vals, y_preds, labels=range(201))

class_acc = []
best = []
worst = []

# Getting accuracies for each label class
for k, v in report.items():
  if(isinstance(int(k), int)):
    if(int(k) == 0):
      acc = 0
    else:
      tp = conf_matrix[int(k)][int(k)]
      num = v.get('support')
      acc = tp/num
      class_acc.append(acc)
      if(k=='200'):
        break

# Getting top 5 and bottom 5 labels
# appending by label, acc, precision, recall, 
for i in range(5):
  maxA = np.argmax(class_acc)
  max_precision = report.get(str(maxA)).get('precision')
  max_recall = report.get(str(maxA)).get('recall')
  max_bcr = (max_precision + max_recall)/2
  best.append((maxA, class_acc[maxA], max_precision, max_recall, max_bcr))
  class_acc.pop(maxA)


  minA = np.argmin(class_acc)
  min_precision = report.get(str(minA)).get('precision')
  min_recall = report.get(str(minA)).get('recall')
  min_bcr = (min_precision + min_recall)/2
  worst.append((minA, class_acc[minA], min_precision, min_recall, min_bcr))
  class_acc.pop(minA)
print('best:', best)
print('worst:', worst)
  


In [ ]:
torch.save(net.state_dict(), '/content/basemodel.pt')

In [ ]:
torch.save(net.state_dict(), '/content/basemodel.pt')